In [ ]:
import numpy as np
from numpy.linalg import svd, norm, matrix_rank, solve, pinv
import scipy.linalg as splg
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import imageio

#-------------------------------------------------------------------------------
# SECTION 1.1 - LEAST SQUARES PROBLEM: Write a program to solve the LS problem 
# using SVD. Compute the LS solution for the datasets datafile and datafile2.csv
# that were used in pr4: QR factorization and least square problems. Compare the
# results using SVD with those obtained from the QR solution of the LS problems.
#-------------------------------------------------------------------------------

'''
In this section the following functions are defined:
- SVD_LS_solve(A, b)
- QR_LS_solve(A, b)
- A_mat(d)
These have been used to obtain the presented outputs
'''

def SVD_LS_solve(A, b):
  # Compute the SVD ------------------------------------------------------------
  #U, s, VH = svd(A)  

  # Compute the Moore-Penrose (left) pseudo-inverse ----------------------------
  #A_pi = (VH.T).dot(np.diag(1/s).dot(U.T))
  A_pi = pinv(A)      # There is already a function implemented in numpy.linalg which computes the pseudo inverse

  # Compute the LS solution with minimum norm ----------------------------------
  x_t = A_pi.dot(b)

  # Compute and display the norm of the solution and the error -----------------
  sol_norm = norm(x_t)
  err = norm(A.dot(x_t)-b)
  print("The obtained solution has norm: ", sol_norm)
  print("It gives an error of: ", err)

  return x_t, sol_norm, err

def QR_LS_solve(A, b):
  # QR decomposition -----------------------------------------------------------
  Q, R, P = splg.qr(A, pivoting=True)
  y = np.dot(Q.T, b)
  
  # If A is full-rank, these lines will have no effect -------------------------
  r = matrix_rank(A)
  n = A.shape[1]
  y1, y2 = y[:r], y[r:]
  R1 = R[:r, :r]

  x = np.concatenate((splg.solve_triangular(R1, y1)[:r], np.zeros(n - r)))

  # Compute the LS solution with minimum norm ----------------------------------
  x_t = solve(np.transpose(np.eye(n)[:,P]), x)

  # Compute and display the norm of the solution and the error -----------------
  sol_norm = norm(x_t)
  err = norm(A.dot(x_t)-b)
  print("The obtained solution has norm: ", sol_norm)
  print("It gives an error of: ", err)

  return x_t, sol_norm, err

# Define a function that returns the matrix A for a given degree d -------------
def A_mat(d):
  """A_mat
  Input: d (a given degree for the polynomial)
  Ouput: A 
  """
  A = np.ones([len(x), d])      # Initialize the matrix A
  for j in range(0, d):
    A[:,j] = x**j
  return A

#-------------------------------------------------------------------------------
# SECTION 2.2 - GRAPHICS COMPRESSION: Use the previous results to obtain a lossy
# compressed graphic image from a .jpeg graphic file. A .jpeg graphic file can 
# be read as a matrix using the function scipy.ndimage.imread(). Use SVD 
# decomposition to create approximations of lower rank to the image. Compare 
# different approximations. The function scipy.misc.imsave() can be useful to 
# save the approximated graphic files as .jpeg. The code must generate different
# compressed files for a given graphic file. Hence, to organize the output 
# files, the name of the compressed file must reflect the percentage of the
# Frobenius norm captured in each compressed file. Use different .jpeg images 
# (of different sizes and having letters or pictures) and compare results.
#-------------------------------------------------------------------------------

'''
In this section the following functions are defined:
- k_rank_approx(img, mode, ks, f_name="NO")
- compression_rate(img, mode)
These have been used to obtain the presented outputs
'''

# Define a function that computes the best k rank approximation of the image ---
def k_rank_approx(img, mode, ks, f_name="NO"):
  """ k_rank_approx
  Inputs: img (grayscale or RGB), mode ("BW" or "RGB"), ks (array of values of k, rank of the approximation)
  Output: None (it shows a preview of the produced images and it saves them as .jpeg files)
  """
  if mode=="BW":
    imggray = img.convert('LA')     # Making sure the image is in a grayscale
    plt.figure()
    plt.imshow(imggray);
    plt.title("Original image")

    # Convert the image to a numpy matrix --------------------------------------
    X = np.array(list(imggray.getdata(band=0)), float)
    X.shape = (imggray.size[1], imggray.size[0])
    X = np.matrix(X)

    # SVD decomposition of the image -------------------------------------------
    U, s, VH = svd(X)     # SVD decomposition

    # Reconstruct the rank k best approximation of the image -------------------
    for k in ks:
      reconstimg = np.matrix(U[:, :k]) * np.diag(s[:k]) * np.matrix(VH[:k, :])

      # Show the image ---------------------------------------------------------
      compression_rate = int(norm(reconstimg-X)/norm(X)*100)
      plt.figure()
      plt.imshow(reconstimg, cmap='gray')
      title = "Compression of rank k=%s" % k + ", rate=" + str(compression_rate) + "%"
      plt.title(title)
      plt.show()

      # Save the image ---------------------------------------------------------
      if f_name != "NO":
        filename = f_name + "rank_" + str(k) + "_compression_rate_" + str(compression_rate) + "%" + ".jpeg"
        imageio.imwrite(filename, reconstimg)
  
  if mode=="RGB":
    plt.figure()
    plt.imshow(img);
    plt.title("Original image")

    red, green, blue = img.split()  # Split the image into channels
    # Convert each channel into matrices ---------------------------------------
    X_r = np.array(list(red.getdata(band=0)), float)
    X_r.shape = (red.size[1], red.size[0])
    X_r = np.matrix(X_r)
    X_r = X_r/255
    X_g = np.array(list(green.getdata(band=0)), float)
    X_g.shape = (green.size[1], green.size[0])
    X_g = np.matrix(X_g)
    X_g = X_g/255
    X_b = np.array(list(blue.getdata(band=0)), float)
    X_b.shape = (blue.size[1], blue.size[0])
    X_b = np.matrix(X_b)
    X_b = X_b/255

    # SVD decomposition of the image -------------------------------------------
    U_r, s_r, VH_r = svd(X_r)     # SVD decomposition for the red channel
    U_g, s_g, VH_g = svd(X_g)     # SVD decomposition for the green channel
    U_b, s_b, VH_b = svd(X_b)     # SVD decomposition for the blue channel

    # Reconstruct the rank k best approximation of each channel ----------------
    for k in ks:
      reconst_red = np.matrix(U_r[:, :k]) * np.diag(s_r[:k]) * np.matrix(VH_r[:k, :])
      reconst_green = np.matrix(U_g[:, :k]) * np.diag(s_g[:k]) * np.matrix(VH_g[:k, :])
      reconst_blue = np.matrix(U_b[:, :k]) * np.diag(s_b[:k]) * np.matrix(VH_b[:k, :])
      img_reconstructed = np.zeros([img.size[1], img.size[0], 3])
      img_reconstructed[:,:,0] = reconst_red
      img_reconstructed[:,:,1] = reconst_green
      img_reconstructed[:,:,2] = reconst_blue

      # Show the image ---------------------------------------------------------
      compression_rate = int((norm(reconst_red-X_r)/norm(X_r)+norm(reconst_green-X_g)/norm(X_g)+norm(reconst_blue-X_b)/norm(X_b))/3*100)
      plt.figure()
      plt.imshow(img_reconstructed);
      title ="Compression of rank k=%s" % k + ", rate=" + str(compression_rate) + "%"
      plt.title(title)
      plt.show()

      # Save the image ---------------------------------------------------------
      if f_name != "NO":
        filename = f_name + "rank_" + str(k) + "_compression_rate_" + str(compression_rate) + "%" + ".jpeg"
        imageio.imwrite(filename, img_reconstructed)

# Define a function to compute the compression rate for all the possible values 
# of the rank k of the compression ---------------------------------------------
def compression_rate(img, mode):
  """compression_rate
  Input: img (grayscale or RGB), mode ("BW" or "RGB")
  Output: None (it shows a plot of the compression rate for all the possible ranks k)
  """
  if mode=="BW":
    imggray = img.convert('LA')     # Making sure the image is in a grayscale
    plt.figure()
    plt.imshow(imggray);
    plt.title("Original image")

    # Convert the image to a numpy matrix --------------------------------------
    X = np.array(list(imggray.getdata(band=0)), float)
    X.shape = (imggray.size[1], imggray.size[0])
    X = np.matrix(X)

    # Initialize vectors -------------------------------------------------------
    ks = np.arange(1, min(X.shape)+1)
    compression_rate = np.zeros([min(X.shape), 1])

    # SVD decomposition of the image -------------------------------------------
    U, s, VH = svd(X)     # SVD decomposition

    # Reconstruct the rank k best approximation of the image -------------------
    for k in ks:
      reconstimg = np.matrix(U[:, :k]) * np.diag(s[:k]) * np.matrix(VH[:k, :])

      # Compute the compression rate -------------------------------------------
      compression_rate[k-1] = (norm(reconstimg-X)/norm(X)*100)

    # Plot the compression rate ------------------------------------------------
    plt.figure()
    plt.plot(ks, compression_rate)
    plt.title("BW Compression rate")
  
  if mode=="RGB":
    plt.figure()
    plt.imshow(img);
    plt.title("Original image")

    red, green, blue = img.split()  # Split the image into channels
    # Convert each channel into matrices ---------------------------------------
    X_r = np.array(list(red.getdata(band=0)), float)
    X_r.shape = (red.size[1], red.size[0])
    X_r = np.matrix(X_r)
    X_r = X_r/255
    X_g = np.array(list(green.getdata(band=0)), float)
    X_g.shape = (green.size[1], green.size[0])
    X_g = np.matrix(X_g)
    X_g = X_g/255
    X_b = np.array(list(blue.getdata(band=0)), float)
    X_b.shape = (blue.size[1], blue.size[0])
    X_b = np.matrix(X_b)
    X_b = X_b/255

    # SVD decomposition of the image -------------------------------------------
    U_r, s_r, VH_r = svd(X_r)     # SVD decomposition for the red channel
    U_g, s_g, VH_g = svd(X_g)     # SVD decomposition for the green channel
    U_b, s_b, VH_b = svd(X_b)     # SVD decomposition for the blue channel

    # Initialize vectors -------------------------------------------------------
    ks = np.arange(1, min(X_r.shape)+1)
    compression_rate = np.zeros([min(X_r.shape), 1])

    # Reconstruct the rank k best approximation of each channel ----------------
    for k in ks:
      reconst_red = np.matrix(U_r[:, :k]) * np.diag(s_r[:k]) * np.matrix(VH_r[:k, :])
      reconst_green = np.matrix(U_g[:, :k]) * np.diag(s_g[:k]) * np.matrix(VH_g[:k, :])
      reconst_blue = np.matrix(U_b[:, :k]) * np.diag(s_b[:k]) * np.matrix(VH_b[:k, :])
      img_reconstructed = np.zeros([img.size[1], img.size[0], 3])
      img_reconstructed[:,:,0] = reconst_red
      img_reconstructed[:,:,1] = reconst_green
      img_reconstructed[:,:,2] = reconst_blue

      # compute the compression rate -------------------------------------------
      compression_rate[k-1] = ((norm(reconst_red-X_r)/norm(X_r)+norm(reconst_green-X_g)/norm(X_g)+norm(reconst_blue-X_b)/norm(X_b))/3*100)

    # Plot the compression rate ------------------------------------------------
    plt.figure()
    plt.plot(ks, compression_rate)
    plt.title("RGB Compression rate")

#-------------------------------------------------------------------------------
# SECTION 3.1 - PRINCIPAL COMPONENT ANALYSIS: The file example.dat contains a 
# dataset of 16 observations of 4 variables. Perform PCA analysis using both the
# covariance matrix and the correlation matrix. The code must write down the 
# portion of the total variance accumulated in each of the principal components,
# the standard deviation of each of the principal components and the expression 
# of the original dataset in the new PCA coordinates.
#-------------------------------------------------------------------------------

'''
In this section the following functions are defined:
- PCA(data, mode, full_matrices=True)
These have been used to obtain the presented outputs
'''

# Define a function to do PCA Analysis on a given dataset ----------------------
def PCA(data, mode, full_matrices=True):
  # Retrieve the dimensions of the problem -------------------------------------
  m = data.shape[0]
  n = data.shape[1]

  # Covariance matrix ----------------------------------------------------------
  if mode=="Covariance":
    # Center the data ----------------------------------------------------------
    means = np.mean(data, axis=0)
    X = np.copy(data)
    idx = 0
    for mean in means:
      X[:,idx] = data[:,idx]-mean
      idx = idx+1
  # Correlation matrix ---------------------------------------------------------
  if mode=="Correlation":
    # Standardize the data -----------------------------------------------------
    means = np.mean(data, axis=0)
    std_devs = np.std(data, axis=0)
    X = np.copy(data)
    idx = 0
    for mean in means:
      X[:,idx] = (data[:,idx]-mean)/std_devs[idx]
      idx = idx+1

  # Analysis -------------------------------------------------------------------
  Y = (1/np.sqrt(n-1))*(X.T)    # Define Y such that Y.T*Y=Cx
  U, s, VH = svd(Y, full_matrices=full_matrices)     # SVD decomposition

  tot_var = np.dot(s, s)
  rel_var = s*s/tot_var*100
  print("s: ", s)
  print("Total variance: ", tot_var)
  print("Relative variance: ", rel_var)

  x_labels = []
  for i in range(0, len(rel_var)):
    x_labels.append("PC"+str(i+1))
  plt.figure()
  plt.bar(x_labels, rel_var)
  plt.title("Scree Plot")

  # Compute the new PCA coordinates --------------------------------------------
  X_new = X.dot(U)

  plt.figure()
  plt.scatter(X_new[:,0], X_new[:,1], alpha=0.4)
  plt.xlabel("PC1: " + str(rel_var[0]) + "% variance")
  plt.ylabel("PC2: " + str(rel_var[1]) + "% variance")

  return X_new, rel_var 

#-------------------------------------------------------------------------------
# SECTION 3.2 - PRINCIPAL COMPONENT ANALYSIS: The file RCsGoff.csv contains data
# from the experiment reported in [3]. Each observation consists in measuring 
# the amount of a total number of 58581 genes. There are a total of 20 
# observations grouped by day of observation. The code must perform a PCA 
# analysis on the covariance matrix. The output file must contain rows with the 
# following format Sample,PC1,PC2,. . . ,PC20,Variance where Sample stands for 
# day0 rep1,...,day18 rep3 (i.e. the different observations) and PCi stands for 
# the coordinate of the principal component of the observation. Finally variance
#  is the portion of the total variance accumulated in each of the principal 
# components. To compare with, below there is the plot of the first two 
# principal components. The memory should include a discussion about the number 
# of principal components needed to explain the data sets (using for example the
# Kaiser rule, Scree plot, the 3/4 of the total variance rule, or any other 
# method; explain the main idea of the methods used).
#-------------------------------------------------------------------------------

'''
In this section the following functions are defined:
- PCA(data, mode, full_matrices=True)
These have been used to obtain the presented outputs
'''

# Define a function to do PCA Analysis on a given dataset ----------------------
def PCA(data, mode, full_matrices=True):
  # Retrieve the dimensions of the problem -------------------------------------
  m = data.shape[0]
  n = data.shape[1]

  # Covariance matrix ----------------------------------------------------------
  if mode=="Covariance":
    # Center the data ----------------------------------------------------------
    means = np.mean(data, axis=0)
    X = np.copy(data)
    idx = 0
    for mean in means:
      X[:,idx] = data[:,idx]-mean
      idx = idx+1
  # Correlation matrix ---------------------------------------------------------
  if mode=="Correlation":
    # Standardize the data -----------------------------------------------------
    means = np.mean(data, axis=0)
    std_devs = np.std(data, axis=0)
    X = np.copy(data)
    idx = 0
    for mean in means:
      X[:,idx] = (data[:,idx]-mean)/std_devs[idx]
      idx = idx+1

  # Analysis -------------------------------------------------------------------
  Y = (1/np.sqrt(n-1))*(X.T)    # Define Y such that Y.T*Y=Cx
  U, s, VH = svd(Y, full_matrices=full_matrices)     # SVD decomposition

  tot_var = np.dot(s, s)
  rel_var = s*s/tot_var*100
  print("s: ", s)
  print("Total variance: ", tot_var)
  print("Relative variance: ", rel_var)

  x_labels = []
  for i in range(0, len(rel_var)):
    x_labels.append("PC"+str(i+1))
  plt.figure()
  plt.bar(x_labels, rel_var)
  plt.title("Scree Plot")

  # Compute the new PCA coordinates --------------------------------------------
  X_new = X.dot(U)

  plt.figure()
  plt.scatter(X_new[:,0], X_new[:,1], alpha=0.4)
  plt.xlabel("PC1: " + str(rel_var[0]) + "% variance")
  plt.ylabel("PC2: " + str(rel_var[1]) + "% variance")

  return X_new, rel_var